In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=92854458c428972e2a87d463d057c0759d8491a0dc10911e83a4f567e1141498
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


Reading file

In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, split, avg, max, min, count

spark = SparkSession.builder \
    .appName("MovieAnalysis") \
    .getOrCreate()

movies_df = spark.read.csv("Movies.csv.csv", header=True, inferSchema=True)

movies_df.show()

+----+------+--------------------+-------+--------------------+-----------------+--------------------+----------+------+-------------------+
|Year|Length|               Title|  Genre|               Actor|          Actress|            Director|Popularity|Awards|              Image|
+----+------+--------------------+-------+--------------------+-----------------+--------------------+----------+------+-------------------+
|1990|   111|Tie Me Up! Tie Me...| Comedy|     BanderasAntonio|    AbrilVictoria|      AlmodóvarPedro|        68|    No|   NicholasCage.png|
|1991|   113|          High Heels| Comedy|          BoséMiguel|    AbrilVictoria|      AlmodóvarPedro|        68|    No|   NicholasCage.png|
|1983|   104|        Dead ZoneThe| Horror|   WalkenChristopher|      AdamsBrooke|     CronenbergDavid|        79|    No|   NicholasCage.png|
|1979|   122|                Cuba| Action|         ConnerySean|      AdamsBrooke|       LesterRichard|         6|    No|    seanConnery.png|
|1978|    94|

Question 1

In [12]:
movie_awards = movies_df.filter((col("Genre") == "Action") & (col("Awards") == "Yes")) \
    .select("Title", "Year", "Director")
movie_awards.show()

+-----+----+--------+
|Title|Year|Director|
+-----+----+--------+
+-----+----+--------+



Question 2

In [13]:
movies_df = movies_df.withColumn("Actor", split(col("Actor"), ", "))
movies_df = movies_df.withColumn("Actress", split(col("Actress"), ", "))
movies_df = movies_df.withColumn("Actor", explode(col("Actor")))
movies_df = movies_df.withColumn("Actress", explode(col("Actress")))
award_winning_movies = movies_df.filter(col("Awards") == "Yes")
actor_movies = award_winning_movies.select("Actor", "Title", "Director")
actor_movies.show(truncate=False)

+---------------------+----------------------------+-------------------+
|Actor                |Title                       |Director           |
+---------------------+----------------------------+-------------------+
|AhlstedtBörje        |Fanny and Alexander         |BergmanIngmar      |
|TrintignantJean-Louis|A Man & a Woman             |LelouchClaude      |
|TrintignantJean-Louis|Un Hombre y una Mujer       |LelouchClaude      |
|AlterioHector        |Official StoryThe           |PuenzoLuiz         |
|SjöströmVictor       |Wild Strawberries           |BergmanIngmar      |
|BjörnstrandGunnar    |Through a Glass Darkly      |BergmanIngmar      |
|JosephsonErland      |Cries & Whispers            |BergmanIngmar      |
|PlummerChristopher   |Sound of MusicThe           |WiseRobert         |
|LaFontJean-Philippe  |Babette's Feast             |AxelGabriel        |
|BalsamMartin         |Murder on the Orient Express|LumetSidney        |
|HopkinsAnthony       |Elephant ManThe             

Question 3

In [14]:
no_awards = movies_df.filter(col("Awards") == "No") \
    .orderBy(col("Popularity").desc()) \
    .select("Title", "Popularity") \
    .limit(10)
no_awards.show(truncate=False)

+---------------------+----------+
|Title                |Popularity|
+---------------------+----------+
|Five Corners         |88        |
|Ballad of NarayamaThe|88        |
|Let It Ride          |88        |
|Final Notice         |88        |
|New Year's Day       |88        |
|Guilty by Suspicion  |88        |
|Fellini Satyricon    |88        |
|Raw Nerve            |88        |
|Time MachineThe      |88        |
|Long Voyage HomeThe  |88        |
+---------------------+----------+



Question 4

In [15]:
rdd4= movies_df.filter(col("Year") < 1980) \
    .orderBy(col("Popularity")) \
    .select("Title", "Popularity") \
    .limit(10)
rdd4.show(truncate=False)

+------------------+----------+
|Title             |Popularity|
+------------------+----------+
|White Lightning   |NULL      |
|Drop KickThe      |NULL      |
|Desert Rider      |NULL      |
|Shalako           |0         |
|Airport           |0         |
|Anna Christie     |0         |
|Shout at the Devil|0         |
|Holocaust         |1         |
|Indiscreet        |1         |
|Stavisky          |1         |
+------------------+----------+



Question 5

In [16]:
average_length = movies_df.groupBy("Genre").agg(avg("Length").alias("AverageLength"))
average_length.show(truncate=False)

+---------------+------------------+
|Genre          |AverageLength     |
+---------------+------------------+
|Crime          |66.0              |
|Romance        |127.0             |
|Adventure      |119.0             |
|NULL           |120.5             |
|Drama          |112.78490566037736|
|War            |112.70588235294117|
|Fantasy        |102.0             |
|Mystery        |104.9390243902439 |
|Music          |115.8076923076923 |
|Science Fiction|106.15625         |
|Horror         |96.33333333333333 |
|Short          |40.0              |
|Western        |108.78846153846153|
|Comedy         |98.85049833887044 |
|Action         |104.41258741258741|
|Westerns       |110.33333333333333|
+---------------+------------------+



Question 6

In [32]:
rdd6 = movies_df.filter(col("genre") == "Comedy").groupBy("actor", "actress").count().filter(col("count") > 3)
rdd6.show()

+------------+----------------+-----+
|       actor|         actress|count|
+------------+----------------+-----+
|TracySpencer|HepburnKatharine|    6|
|  AllenWoody|     KeatonDiane|    5|
+------------+----------------+-----+



Question 7

In [18]:
rdd7 = movies_df.filter(col("Genre") == "Comedy") \
    .select("Actor")
drama_actors = movies_df.filter(col("Genre") == "Drama") \
    .select("Actor")
common_actors = rdd7.intersect(drama_actors)
common_actors.show(truncate=False)

+---------------+
|Actor          |
+---------------+
|WillisBruce    |
|IronsJeremy    |
|EastwoodClint  |
|ConnerySean    |
|TracySpencer   |
|AielloDanny    |
|MooreDudley    |
|QuinnAidan     |
|AdolphsonEdvin |
|FinchPeter     |
|BrandoMarlon   |
|NewmanPaul     |
|BeattyWarren   |
|CaanJames      |
|HowellC. Thomas|
|RogersWill     |
|CageNicolas    |
|RedfordRobert  |
|CaineMichael   |
|O'ToolePeter   |
+---------------+
only showing top 20 rows



Question 8

In [19]:
intersection = comedy_actors.union(drama_actors)
intersection.show(truncate=False)

+---------------+
|Actor          |
+---------------+
|BanderasAntonio|
|BoséMiguel     |
|WayneJohn      |
|MurrayBill     |
|NivenDavid     |
|MooreDudley    |
|ReynoldsBurt   |
|CleeseJohn     |
|DepardieuGérard|
|AldaAlan       |
|RobertsEric    |
|DunneGriffin   |
|QuinnAidan     |
|MatthauWalter  |
|GelinDaniel    |
|BretonniereJean|
|VidalHenri     |
|MumyBilly      |
|PerkinsAnthony |
|JourdanLouis   |
+---------------+
only showing top 20 rows



Question 9

In [20]:
rdd9 = movies_df.filter(col("Genre") != "Comedy") \
    .select("Actor") \
    .distinct()
rdd9.show(truncate=False)

+---------------+
|Actor          |
+---------------+
|CottenJoseph   |
|BrownTom       |
|DillonMatt     |
|LancasterBurt  |
|WillisBruce    |
|RomeroCesar    |
|UrichRobert    |
|DavisGuy       |
|BridgesBeau    |
|KattWilliam    |
|EnglundRobert  |
|PriceMarc      |
|CapolicchioLino|
|FondaPeter     |
|TownsendRobert |
|ChesneyArthur  |
|BoyceAlan      |
|WidgrenOlof    |
|IronsJeremy    |
|PayneJohn      |
+---------------+
only showing top 20 rows



Question 10

In [21]:
rdd10= movies_df.groupBy("Actor").agg(avg("Popularity").alias("MeanRanking"),
                                                max("Popularity").alias("MaxRanking"),
                                                min("Popularity").alias("MinRanking"))
rdd10.show(truncate=False)

+---------------+------------------+----------+----------+
|Actor          |MeanRanking       |MaxRanking|MinRanking|
+---------------+------------------+----------+----------+
|BoséMiguel     |68.0              |68        |68        |
|CottenJoseph   |58.0              |74        |32        |
|BrownTom       |77.0              |77        |77        |
|DillonMatt     |7.5               |11        |4         |
|KeatonMichael  |59.0              |59        |59        |
|WillisBruce    |48.0              |76        |7         |
|LancasterBurt  |40.083333333333336|84        |0         |
|RomeroCesar    |78.0              |78        |78        |
|JourdanLouis   |70.0              |70        |70        |
|ModineMatthew  |8.0               |8         |8         |
|UrichRobert    |64.0              |64        |64        |
|JaglomHenry    |88.0              |88        |88        |
|DavisGuy       |72.0              |72        |72        |
|BridgesBeau    |64.0              |64        |64       

Question 11

In [29]:
rdd11= movies_df.withColumn("Decade", ((col("Year") - 1960) / 10).cast("int") * 10 + 1960)
rdd11 = movies_df.groupBy("Decade").agg(count("*").alias("MoviesCount"))
rdd11.show(truncate=False)


+------+-----------+
|Decade|MoviesCount|
+------+-----------+
|1990  |237        |
|1930  |21         |
|1950  |65         |
|1960  |207        |
|1970  |193        |
|1980  |499        |
|1940  |53         |
+------+-----------+



Question 12

In [23]:
rdd12 = movies_df.groupBy("Year").agg(count("*").alias("MoviesCount"))
rdd12.show(truncate=False)

+----+-----------+
|Year|MoviesCount|
+----+-----------+
|1959|7          |
|1990|78         |
|1975|17         |
|1977|26         |
|1924|2          |
|1974|19         |
|1927|3          |
|1955|17         |
|1978|17         |
|1925|1          |
|1961|11         |
|1942|4          |
|1944|2          |
|1939|8          |
|1952|7          |
|1956|12         |
|1934|2          |
|1988|84         |
|1997|1          |
|1968|17         |
+----+-----------+
only showing top 20 rows



Question 13

In [24]:
movies_df_filtered = movies_df.filter(col("Length") > 100)
rdd13 = movies_df_filtered.groupBy("Year", "Genre").agg(count("*").alias("MoviesCount"))
rdd13.show(truncate=False)

+----+---------------+-----------+
|Year|Genre          |MoviesCount|
+----+---------------+-----------+
|1989|Action         |9          |
|1988|Comedy         |10         |
|1966|Drama          |5          |
|1985|Comedy         |4          |
|1957|Western        |1          |
|1928|Drama          |1          |
|1970|Western        |1          |
|1975|Drama          |4          |
|1969|Action         |1          |
|1940|Action         |1          |
|1939|Western        |1          |
|1987|Comedy         |12         |
|1982|Science Fiction|1          |
|1951|War            |1          |
|1965|Music          |1          |
|1939|Comedy         |1          |
|1964|Music          |1          |
|1987|War            |1          |
|1966|Mystery        |2          |
|1946|Mystery        |1          |
+----+---------------+-----------+
only showing top 20 rows



Question 14

In [25]:
rdd14 = movies_df.filter(col("Year") < 1990) \
    .orderBy(col("Title"))
rdd14.show(truncate=False)

+----+------+-----------------------------------+---------------+---------------------+-----------------+-------------------+----------+------+-------------------+------+
|Year|Length|Title                              |Genre          |Actor                |Actress          |Director           |Popularity|Awards|Image              |Decade|
+----+------+-----------------------------------+---------------+---------------------+-----------------+-------------------+----------+------+-------------------+------+
|1968|139   |2001: A Space Odyssey              |Science Fiction|DulleaKeir           |TyzackMargaret   |KubrickStanley     |83        |No    |NicholasCage.png   |1960  |
|1982|92    |48 Hrs.                            |Action         |NolteNick            |O'TooleAnnette   |HillWalter         |67        |No    |NicholasCage.png   |1980  |
|1963|138   |8 1/2                              |Drama          |MastroianniMarcello  |CardinaleClaudia |FelliniFederico    |80        |Yes   |Ni

Question 15

In [26]:
rdd15 = movies_df.filter(col("Title").rlike("[a-zA-Z]{51,}"))
rdd15.show(truncate=False)

+----+------+-----+-----+-----+-------+--------+----------+------+-----+------+
|Year|Length|Title|Genre|Actor|Actress|Director|Popularity|Awards|Image|Decade|
+----+------+-----+-----+-----+-------+--------+----------+------+-----+------+
+----+------+-----+-----+-----+-------+--------+----------+------+-----+------+

